In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pyspedas
import cdflib

<h2 style='color:cyan'>Load in Spacecraft Data</h2>

The first assignment is going to be loading in spacecraft data from Parker Solar Probe. When we discuss solar wind turbulence we need to understand the couplings between the velocity and magnetic field fluctuations. Therefore, we are going to need to load in both the magnetic field and the velocity data. We have several options on how to load in spacecraft data. We can load the data directly from a url based server (i.e. CDAweb), the instrument team websites (i.e., FIELDS, SWEAP). However, in python there are several opensource packages that will grab the data and load it into your local directory such as PySPEDAS, CDAweb (python package), and SUNPY.

Below is a simple script loading in the magnetic field and solar wind plasma data for Parker Solar Probe, using PySPEDAS. 

In [3]:
trange = ['2024-03-27T00:00:00', '2024-03-27T23:59:00']

# Get the magnetic field data from pyspedas.
psp_fields = pyspedas.psp.fields(trange, 
                                 datatype='mag_SC_4_Sa_per_Cyc',    # This is the downsampled 4 samples per NY second (0.87 s = 1 NYs) data. You will load in the high resolution data later!
                                 level='l2',                        # l2 is the first level of public data that is available to the public. 
                                 notplot=True,                      # You can play around with this if you want to use the tplot functions. I do not like tplot values.
                                 downloadonly=True,                 # Since we are not using the tplot values, and we do not want to read the data with pyspedas I always set this to be True.
                                 last_version=True,                 # Ensures we are looking at the most up to data data product
                                 get_support_data=False)#,          # Loads in additional support data, only useful for certain PSP data products. Default is false.
                                 # username=username,           If we go to the latest Parker Solar Probe encounter we can give you a username and password
                                 # password=password)

# Get the solar wind data from pyspedas.
psp_spc = pyspedas.psp.spc(trange, 
                           datatype='l3i',
                           level='l3',
                           notplot=True,
                           downloadonly=True,
                           last_version=True,
                           get_support_data=False)



06-Jun-25 18:00:32: Downloading remote index: https://spdf.gsfc.nasa.gov/pub/data/psp/fields/l2/mag_sc_4_per_cycle/2024/
06-Jun-25 18:00:32: Downloading https://spdf.gsfc.nasa.gov/pub/data/psp/fields/l2/mag_sc_4_per_cycle/2024/psp_fld_l2_mag_sc_4_sa_per_cyc_20240327_v02.cdf to psp_data/fields/l2/mag_sc_4_per_cycle/2024/psp_fld_l2_mag_sc_4_sa_per_cyc_20240327_v02.cdf
06-Jun-25 18:00:34: Download complete: psp_data/fields/l2/mag_sc_4_per_cycle/2024/psp_fld_l2_mag_sc_4_sa_per_cyc_20240327_v02.cdf
06-Jun-25 18:00:34: Downloading remote index: https://spdf.gsfc.nasa.gov/pub/data/psp/sweap/spc/l3/l3i/2024/
06-Jun-25 18:00:35: Downloading https://spdf.gsfc.nasa.gov/pub/data/psp/sweap/spc/l3/l3i/2024/psp_swp_spc_l3i_20240327_v02.cdf to psp_data/sweap/spc/l3/l3i/2024/psp_swp_spc_l3i_20240327_v02.cdf


Using LEVEL=L3


06-Jun-25 18:00:45: Download complete: psp_data/sweap/spc/l3/l3i/2024/psp_swp_spc_l3i_20240327_v02.cdf


In [4]:
# Check that the files are loaded in
print(psp_fields)
print(psp_spc)

['psp_data/fields/l2/mag_sc_4_per_cycle/2024/psp_fld_l2_mag_sc_4_sa_per_cyc_20240327_v02.cdf']
['psp_data/sweap/spc/l3/l3i/2024/psp_swp_spc_l3i_20240327_v02.cdf']


Notice, that the scripts above generated a local file called ./psp_data. This is the default behavior and all subsequent calls will store data in this directory. I do not use pyspedas to work with data, as it used a pytplot format that is slightly cumbersome to work with, hence, I only use it to load in the data.   

Now that the files are loaded into our directory, I will use cdflib to read the selected files. 

In [11]:
# Now that we have the filenames loaded in we can use cdflib.xarray to load in the datasets.
fields_xr = cdflib.xarray.cdf_to_xarray(psp_fields[0])
spc_xr    = cdflib.xarray.cdf_to_xarray(psp_spc[0])

In [12]:
# This lets you see what the xarray data format looks like in an interactive cell.
fields_xr

<xarray.Dataset> Size: 8MB
Dimensions:                         (epoch_mag_SC_4_Sa_per_Cyc: 395507,
                                     component_index_SC: 3,
                                     epoch_mag_SC_zero: 1440, dim0: 3, dim1: 4,
                                     epoch_quality_flags: 1440)
Coordinates:
  * epoch_mag_SC_4_Sa_per_Cyc       (epoch_mag_SC_4_Sa_per_Cyc) datetime64[ns] 3MB ...
  * epoch_mag_SC_zero               (epoch_mag_SC_zero) datetime64[ns] 12kB 2...
  * component_index_SC              (component_index_SC) int32 12B 1 2 3
    label_SC                        (component_index_SC) <U3 36B 'B_X' ... 'B_Z'
  * epoch_quality_flags             (epoch_quality_flags) datetime64[ns] 12kB ...
Dimensions without coordinates: dim0, dim1
Data variables:
    psp_fld_l2_mag_SC_4_Sa_per_Cyc  (epoch_mag_SC_4_Sa_per_Cyc, component_index_SC) float32 5MB ...
    psp_fld_l2_mag_SC_zero          (epoch_mag_SC_zero, dim0, dim1) float32 69kB ...
    psp_fld_l2_quality_flags        (epoch_quality_flags) uint32 6kB 0 0 ... 0 0
Attributes: (12/29)
    TITLE:                         ['PSP FIELDS Fluxgate Magnetometer (MAG) d...
    Project:                       ['PSP']
    Discipline:                    ['Solar Physics>Heliospheric Physics', 'Sp...
    Source_name:                   ['PSP_FLD>Parker Solar Probe FIELDS']
    Descriptor:                    ['MAG_SC_4_Sa_per_Cyc>Fluxgate Magnetomete...
    Data_type:                     ['L2>Level 2 Data']
    ...                            ...
    Generation_date:               ['Sat Jun  1 02:17:04 2024']
    HTTP_LINK:                     ['http://fields.ssl.berkeley.edu/data/']
    LINK_TEXT:                     ['PSP/FIELDS SOC']
    LINK_TITLE:                    ['PSP/FIELDS MAG data available at']
    Time_resolution:               ['4 samples per cycle']
    svn_version:                   ['33364']

In [13]:
# The same for SPC l3 data.
spc_xr

<xarray.Dataset> Size: 82MB
Dimensions:                         (Epoch: 190288, array32_index: 32,
                                     vector_index: 3, dim0: 3, dim1: 32)
Coordinates: (12/51)
  * Epoch                           (Epoch) datetime64[ns] 2MB 2024-03-27T00:...
    np_fit_uncertainty              (Epoch) float32 761kB 93.38 ... 18.15
    wp_fit_uncertainty              (Epoch) float32 761kB 1.058 1.438 ... 2.541
    vp_fit_SC_uncertainty           (Epoch, vector_index) float32 2MB 31.37 ....
    vp_fit_RTN_uncertainty          (Epoch, vector_index) float32 2MB 0.9257 ...
    np1_fit_uncertainty             (Epoch) float32 761kB 24.1 8.011 ... 18.01
    ...                              ...
    v3_fit_SC_uncertainty_LABL_1    (vector_index) <U22 264B 'v3_fit_xSC_unce...
    v3_fit_RTN_LABL_1               (vector_index) <U8 96B 'v3_fit_R' ... 'v3...
    v3_fit_RTN_uncertainty_LABL_1   (vector_index) <U20 240B 'v3_fit_R_uncert...
    HCI_LABL                        (vector_index) <U5 60B 'x_HCI' ... 'z_HCI'
    VHECI_LABL                      (vector_index) <U6 72B 'Vx_HCI' ... 'Vz_HCI'
  * array32_index                   (array32_index) uint8 32B 255 255 ... 255
Dimensions without coordinates: dim0, dim1
Data variables: (12/31)
    DQF                             (Epoch, array32_index) int8 6MB 1 1 ... -1
    np_fit                          (Epoch) float32 761kB 215.1 204.8 ... 368.5
    wp_fit                          (Epoch) float32 761kB 66.46 64.44 ... 52.95
    vp_fit_SC                       (Epoch, vector_index) float32 2MB -99.24 ...
    vp_fit_RTN                      (Epoch, vector_index) float32 2MB 257.2 ....
    np1_fit                         (Epoch) float32 761kB 215.1 204.8 ... 368.5
    ...                              ...
    carr_longitude                  (Epoch) float32 761kB 15.25 15.25 ... 13.85
    SC_frame                        (dim0) <U4 48B 'X_SC' 'Y_SC' 'Z_SC'
    unit_velocity                   (dim0) <U4 48B 'km/s' 'km/s' 'km/s'
    RTN_FRAME                       (dim0) <U1 12B 'R' 'T' 'N'
    DQF_flagnames                   (dim1) <U70 9kB '(0) general flag        ...
    general_flag                    (Epoch) int8 190kB 1 0 0 0 0 0 ... 0 0 0 0 0
Attributes: (12/25)
    Project:                     ['LWS>Living With a Star']
    Source_name:                 ['PSP>Parker Solar Probe']
    Discipline:                  ['Solar Physics>Heliospheric Physics', 'Spac...
    Data_type:                   ['L3i>Solar Wind Ion velocity moments and fi...
    Descriptor:                  ['SWP_SPC>Solar Wind Electrons Alphas and Pr...
    File_naming_convention:      ['source_descriptor_datatype_yyyyMMdd']
    ...                          ...
    Acknowledgement:             ['This project acknowledges the Parker Solar...
    MODS:                        ['2018-12-04: CDF skeleton created (MLS)', '...
    LINK_TITLE:                  ['SWEAP instrument paper', 'Smithsonian Astr...
    HTTP_LINK:                   ['https://link.springer.com/article/10.1007/...
    spase_DatasetResourceID:     ['spase://VSO/NumericalData/LWS/PSP/SPC/TBD']
    Calibrations_used:           ['psp_swp_spc_calfiles_sensorgainoffsets_201...

In [ ]:
# To access a variable we call each individual DataArray in each DataSet
time = fields_xr.epoch_mag_SC_4_Sa_per_Cyc      # Note this keeps the DataArray format

# Note, it is sometimes frustrating to use the default naming conventions therefore we can use the rename property of xarrays
ex_fields_xr = fields_xr.copy()                                 # Working in this convention making a copy is important to ensure you do not overide the orignal copy.
ex_fields_xr.rename({'epoch_mag_SC_4_Sa_per_Cyc' : 'time'})     # This updates all the key information in the xarray.

<xarray.Dataset> Size: 8MB
Dimensions:                         (time: 395507, component_index_SC: 3,
                                     epoch_mag_SC_zero: 1440, dim0: 3, dim1: 4,
                                     epoch_quality_flags: 1440)
Coordinates:
  * time                            (time) datetime64[ns] 3MB 2024-03-27T00:0...
  * epoch_mag_SC_zero               (epoch_mag_SC_zero) datetime64[ns] 12kB 2...
  * component_index_SC              (component_index_SC) int32 12B 1 2 3
    label_SC                        (component_index_SC) <U3 36B 'B_X' ... 'B_Z'
  * epoch_quality_flags             (epoch_quality_flags) datetime64[ns] 12kB ...
Dimensions without coordinates: dim0, dim1
Data variables:
    psp_fld_l2_mag_SC_4_Sa_per_Cyc  (time, component_index_SC) float32 5MB 41...
    psp_fld_l2_mag_SC_zero          (epoch_mag_SC_zero, dim0, dim1) float32 69kB ...
    psp_fld_l2_quality_flags        (epoch_quality_flags) uint32 6kB 0 0 ... 0 0
Attributes: (12/29)
    TITLE:                         ['PSP FIELDS Fluxgate Magnetometer (MAG) d...
    Project:                       ['PSP']
    Discipline:                    ['Solar Physics>Heliospheric Physics', 'Sp...
    Source_name:                   ['PSP_FLD>Parker Solar Probe FIELDS']
    Descriptor:                    ['MAG_SC_4_Sa_per_Cyc>Fluxgate Magnetomete...
    Data_type:                     ['L2>Level 2 Data']
    ...                            ...
    Generation_date:               ['Sat Jun  1 02:17:04 2024']
    HTTP_LINK:                     ['http://fields.ssl.berkeley.edu/data/']
    LINK_TEXT:                     ['PSP/FIELDS SOC']
    LINK_TITLE:                    ['PSP/FIELDS MAG data available at']
    Time_resolution:               ['4 samples per cycle']
    svn_version:                   ['33364']

In [15]:
time

<xarray.DataArray 'epoch_mag_SC_4_Sa_per_Cyc' (epoch_mag_SC_4_Sa_per_Cyc: 395507)> Size: 3MB
array(['2024-03-27T00:00:00.002310272', '2024-03-27T00:00:00.220763648',
       '2024-03-27T00:00:00.439216768', ..., '2024-03-27T23:59:59.416437120',
       '2024-03-27T23:59:59.634890496', '2024-03-27T23:59:59.853343616'],
      dtype='datetime64[ns]')
Coordinates:
  * epoch_mag_SC_4_Sa_per_Cyc  (epoch_mag_SC_4_Sa_per_Cyc) datetime64[ns] 3MB ...
Attributes: (12/20)
    FIELDNAM:            epoch_mag_SC_4_Sa_per_Cyc
    MONOTON:             INCREASE
    FORMAT:              I22
    LABLAXIS:            Epoch
    VAR_TYPE:            support_data
    FILLVAL:             ['NaT']
    ...                  ...
    TIME_SCALE:          Terrestrial Time
    REFERENCE_POSITION:  Rotating Earth Geoid
    TIME_ATTRS:          ['FILLVAL', 'VALIDMIN', 'VALIDMAX', 'SCALEMIN', 'SCA...
    standard_name:       epoch_mag_SC_4_Sa_per_Cyc
    long_name:           Epoch
    units:               Datetime (UTC)

In [ ]:
# To access the data values only we add .data
print(time.data)        # This returns the 1D numpy array for time

['2024-03-27T00:00:00.002310272' '2024-03-27T00:00:00.220763648'
 '2024-03-27T00:00:00.439216768' ... '2024-03-27T23:59:59.416437120'
 '2024-03-27T23:59:59.634890496' '2024-03-27T23:59:59.853343616']


In [18]:
# The wonderful thing about xarrays is we can keep the data attributes stored with the data products. 
# In the example two cells above, notice there is an attributes tab
# We can access those attributes by doing the following.
time.attrs      # This will list all the key properties in the given Data Array. This tells you the fill values, valid mins and maxs, units, and any dependencies.

{'FIELDNAM': 'epoch_mag_SC_4_Sa_per_Cyc',
 'MONOTON': 'INCREASE',
 'FORMAT': 'I22',
 'LABLAXIS': 'Epoch',
 'VAR_TYPE': 'support_data',
 'FILLVAL': array(['NaT'], dtype='datetime64[ns]'),
 'VALIDMIN': array(['2010-01-01T00:00:00.000000000'], dtype='datetime64[ns]'),
 'VALIDMAX': array(['2049-12-31T23:59:59.999999999'], dtype='datetime64[ns]'),
 'SCALEMIN': array(['2024-03-27T00:00:00.000000000'], dtype='datetime64[ns]'),
 'SCALEMAX': array(['2024-03-28T00:00:00.000000000'], dtype='datetime64[ns]'),
 'UNITS': 'Datetime (UTC)',
 'CATDESC': 'Time in TT2000 for 4 samples per cycle cadence MAG waveform data',
 'TIME_BASE': 'J2000',
 'SCALETYP': 'linear',
 'TIME_SCALE': 'Terrestrial Time',
 'REFERENCE_POSITION': 'Rotating Earth Geoid',
 'TIME_ATTRS': ['FILLVAL', 'VALIDMIN', 'VALIDMAX', 'SCALEMIN', 'SCALEMAX'],
 'standard_name': 'epoch_mag_SC_4_Sa_per_Cyc',
 'long_name': 'Epoch',
 'units': 'Datetime (UTC)'}

<h4>Task #0</h4>

Explore xarray to get the different parameters. Report the units for magnetic field, density, velocity, etc. More info on Xarray properties can be found here: https://docs.xarray.dev/en/stable/

Store magnetic field vector, velocity vector, density, and temperature into numpy arrays. Create two time arrays one for the plasma data and one for the mangetic field data. 

<h4>Task #1</h4>

Write a **function(s)** that takes in a time range and will load in both Parker Solar Probe instrument data
1. level 2 magnetic field data. **'mag_SC'** and **'mag_SC_4_Sa_per_Cyc'**
2. level 3 plasma data. **'l3i'**

Try to have one script that will load data into local directory and one function to read the data into a user friendly format. Use the **cdflib** or other package to do the data loading. (It is recommended to use the **cdflib.xarray** packages to store the data locally, see the example below).

Note: We will begin by using **spc l3** data for investigating the fast radial scans. So start with that for now. 

<h4>Task #2</h4>

There are several key parameters solar wind properties that we will be using in this study. To help get an intuitive understanding:

1. **Write the equations** for each parameter listed below.
2. **Answer the conceptual questions** for each one.
3. **Compute each parameter** using your provided time series data.
4. **Visualize** these parameters in a clear, multi-panel plot.

---
1. **Alfvén Speed**: What is the the $Alfv\'{e}n$ speed? How does this paremeter evolve in the solar wind?   

2. **Plasma Beta ($\beta_{p}$)**: What does this parameter represent? What does a high (low) $\beta$ parameter mean? 

3. **Alfvén Mach Number ($M_{A}$)**: Why would we be interested in the $Alfv\'{e}n$ Mach number? What information does this tell us?

4. **$\theta_{vb}$** the angle between velocity and magnetic field vector: Why might this parameter be important?  


---

### Computational Task

Use your time series data to calculate the above plasma parameters. Then:

- Create a **multi-panel plot** (using `matplotlib` or `plotly`) that displays the following:

  - Magnetic field vector components and magnitude
  - Proton velocity vector components and magnitude
  - Proton number density
  - Proton temperature
  - The four derived plasma parameters listed above

---

### Interpolation Step

The plasma and magnetic field measurements will likely have **different time resolutions**. To combine these data:

- Use `numpy.interp` or `scipy.interpolate.interp1d` to **linearly interpolate** one time series onto the other's time base.

- **Which time resolution should you interpolate to?**
  - Should you interpolate the magnetic field to the plasma cadence, or vice versa?
  - Justify your choice.

- **What are the potential pitfalls** of interpolating the wrong way?


<h4>Task #3</h4>

Now we will analyze the **velocity** and **magnetic field** data by separating each signal into its average and fluctuating components. Once the fluctuations are isolated, we will compute the **Power Spectral Density (PSD)** from the autocorrelation function.

For both the velocity and magnetic field time series:
- Compute the average (mean) value of the signal.
- Subtract the mean from the original signal to obtain the **fluctuation** component.

Once the fluctuations are calculated you should calculate the Power Spectral Density.

---
The autocorrelation function for a continuous signal is defined as:

$$
R_{xx}(\tau) = \langle x(t) x(t + \tau) \rangle \approx \frac{1}{T} \int_0^T x(t)\, x(t + \tau)\, dt
$$

Since we are working with discrete data, approximate this by computing the autocorrelation of the fluctuations using:

- `numpy.correlate` or `scipy.signal.correlate` with `mode='full'`. You should plot this to get an idea of what the autocorrelation function is.
- Normalize the result so that \( R_{xx}(0) = 1 \), or use a normalization appropriate for your data.

The Power Spectral Density (PSD) is defined as the **Fourier Transform** of the autocorrelation function:

$$
P_{xx}(f) = \mathcal{F}\left[ R_{xx}(\tau) \right]
$$

Use either `numpy.fft.fft` or `scipy.fft.fft` to compute the **Discrete Fourier Transform (DFT)** of the autocorrelation function. You may use `fftfreq` to interpret the frequency axis. 

---

Compute the normalized power spectrum as:

$$
\text{Power}(f) = \frac{|P_{xx}(f)|^2}{N}
$$

where:
- \( |P_{xx}(f)| \) is the magnitude of the Fourier-transformed autocorrelation,
- \( N \) is the number of points in your original fluctuation time series.

This gives the power associated with each frequency in your signal.

Make a plot showing the Power vs. Frequency in logarithmic space (for both the power and frequency). 

---

To obtain the **trace** (i.e., the total power at each frequency summed over all three vector components), simply sum the power spectra of the individual components:

$$
\text{Trace}(f) = \text{Power}_x(f) + \text{Power}_y(f) + \text{Power}_z(f)
$$

This gives the scalar **total power spectral density** as a function of frequency.

Compare the **trace** with the power spectrum of the **velocity** and **magnetic field** magnitudes. (i.e. PSD(|V|) and PSD(|B|)). Make sure you detrend the data by subtracting the mean.

---

**Note:** This approach relies on the **Wiener–Khinchin theorem**, which relates the autocorrelation function to the power spectral density via the Fourier Transform. We will discuss these details in a future meeting. 
